In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import *

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.kas.layers import Dense,Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
ann=list(set(train.Policy_Sales_Channel).intersection(test.Policy_Sales_Channel))
train['Policy_Sales_Channel']=np.where(train['Policy_Sales_Channel'].isin(ann),train['Policy_Sales_Channel'],'missing')
test['Policy_Sales_Channel']=np.where(test['Policy_Sales_Channel'].isin(ann),test['Policy_Sales_Channel'],'missing')

train.drop(['id'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)

In [4]:
data=pd.concat([train,test])

data['total_policy_in_region']=data.groupby(['Region_Code'])['Policy_Sales_Channel'].transform('nunique')
data['total_region_covered_by_policy']=data.groupby(['Policy_Sales_Channel'])['Region_Code'].transform('nunique')
data['total_policy_in_region']=data['total_policy_in_region'].astype(float)
data['total_region_covered_by_policy']=data['total_region_covered_by_policy'].astype(float)

In [5]:
cols=['Gender','Driving_License','Region_Code','Previously_Insured', 'Vehicle_Age','Vehicle_Damage']
df=data.groupby(cols)['Annual_Premium'].describe().reset_index()
cols2=cols+['25%','50%','75%']
df=df[cols2]
data=pd.merge(data,df,on=cols,how='left')

In [6]:
df=data.groupby(['Policy_Sales_Channel'])['Annual_Premium'].describe().reset_index()
cols=['Policy_Sales_Channel','25%','50%','75%']
df=df[cols]
df.columns=['Policy_Sales_Channel','Policy25%','Policy50%','Policy75%']
data=pd.merge(data,df,on='Policy_Sales_Channel',how='left')

In [7]:
data.drop(['Policy_Sales_Channel'],axis=1,inplace=True)

In [8]:
cat_col=['Gender', 'Driving_License', 'Region_Code','Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']
data=pd.get_dummies(columns=cat_col,data=data)

train2=data[~data['Response'].isna()]
test2=data[data['Response'].isna()]

In [9]:
x=train2.drop(['Response'],axis=1)
y=train2['Response']
test2=test2[x.columns]

In [10]:
train2.head()

,Age,Annual_Premium,Vintage,Response,total_policy_in_region,total_region_covered_by_policy,25%,50%,75%,Policy25%,Policy50%,Policy75%,Gender_Female,Gender_Male,Driving_License_0,Driving_License_1,Region_Code_0.0,Region_Code_1.0,Region_Code_2.0,Region_Code_3.0,Region_Code_4.0,Region_Code_5.0,Region_Code_6.0,Region_Code_7.0,Region_Code_8.0,Region_Code_9.0,Region_Code_10.0,Region_Code_11.0,Region_Code_12.0,Region_Code_13.0,Region_Code_14.0,Region_Code_15.0,Region_Code_16.0,Region_Code_17.0,Region_Code_18.0,Region_Code_19.0,Region_Code_20.0,Region_Code_21.0,Region_Code_22.0,Region_Code_23.0,Region_Code_24.0,Region_Code_25.0,Region_Code_26.0,Region_Code_27.0,Region_Code_28.0,Region_Code_29.0,Region_Code_30.0,Region_Code_31.0,Region_Code_32.0,Region_Code_33.0,Region_Code_34.0,Region_Code_35.0,Region_Code_36.0,Region_Code_37.0,Region_Code_38.0,Region_Code_39.0,Region_Code_40.0,Region_Code_41.0,Region_Code_42.0,Region_Code_43.0,Region_Code_44.0,Region_Code_45.0,Region_Code_46.0,Region_Code_47.0,Region_Code_48.0,Region_Code_49.0,Region_Code_50.0,Region_Code_51.0,Region_Code_52.0,Previously_Insured_0,Previously_Insured_1,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
0,44,40454.0,217,1.0,122.0,53.0,33720.5,41805.5,50110.75,26791.5,34630.0,43177.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
1,76,33536.0,183,0.0,64.0,53.0,2630.0,25222.0,32551.00,26791.5,34630.0,43177.0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
2,47,38294.0,27,1.0,122.0,53.0,33720.5,41805.5,50110.75,26791.5,34630.0,43177.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
3,21,28619.0,203,0.0,53.0,53.0,25049.0,29587.0,34836.50,25324.5,30756.0,37159.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
4,29,27496.0,39,0.0,68.0,53.0,25911.0,31076.0,37237.00,25324.5,30756.0,37159.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0


In [19]:
x.head()

,Age,Annual_Premium,Vintage,total_policy_in_region,total_region_covered_by_policy,25%,50%,75%,Policy25%,Policy50%,Policy75%,Gender_Female,Gender_Male,Driving_License_0,Driving_License_1,Region_Code_0.0,Region_Code_1.0,Region_Code_2.0,Region_Code_3.0,Region_Code_4.0,Region_Code_5.0,Region_Code_6.0,Region_Code_7.0,Region_Code_8.0,Region_Code_9.0,Region_Code_10.0,Region_Code_11.0,Region_Code_12.0,Region_Code_13.0,Region_Code_14.0,Region_Code_15.0,Region_Code_16.0,Region_Code_17.0,Region_Code_18.0,Region_Code_19.0,Region_Code_20.0,Region_Code_21.0,Region_Code_22.0,Region_Code_23.0,Region_Code_24.0,Region_Code_25.0,Region_Code_26.0,Region_Code_27.0,Region_Code_28.0,Region_Code_29.0,Region_Code_30.0,Region_Code_31.0,Region_Code_32.0,Region_Code_33.0,Region_Code_34.0,Region_Code_35.0,Region_Code_36.0,Region_Code_37.0,Region_Code_38.0,Region_Code_39.0,Region_Code_40.0,Region_Code_41.0,Region_Code_42.0,Region_Code_43.0,Region_Code_44.0,Region_Code_45.0,Region_Code_46.0,Region_Code_47.0,Region_Code_48.0,Region_Code_49.0,Region_Code_50.0,Region_Code_51.0,Region_Code_52.0,Previously_Insured_0,Previously_Insured_1,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
0,44,40454.0,217,122.0,53.0,33720.5,41805.5,50110.75,26791.5,34630.0,43177.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
1,76,33536.0,183,64.0,53.0,2630.0,25222.0,32551.00,26791.5,34630.0,43177.0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
2,47,38294.0,27,122.0,53.0,33720.5,41805.5,50110.75,26791.5,34630.0,43177.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
3,21,28619.0,203,53.0,53.0,25049.0,29587.0,34836.50,25324.5,30756.0,37159.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
4,29,27496.0,39,68.0,53.0,25911.0,31076.0,37237.00,25324.5,30756.0,37159.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0


In [12]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10,monitor='val_loss',mode='auto'),
    tf.keras.callbacks.ModelCheckpoint( monitor='val_accuracy',mode='max',save_best_only=True,filepath='best.h5')
]

model = Sequential()
model.add(Dense(32, input_dim = len(x.columns) , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(x,y,batch_size=2000,epochs=50,verbose=True,callbacks=my_callbacks,
          validation_split=0.2,shuffle=True)

Epoch 1/50
153/153 [==============================] - 24s 159ms/step - loss: 158.2361 - accuracy: 0.7946 - val_loss: 0.3862 - val_accuracy: 0.8790
Epoch 2/50
153/153 [==============================] - 1s 4ms/step - loss: 1.6503 - accuracy: 0.8663 - val_loss: 0.4993 - val_accuracy: 0.8790
Epoch 3/50
153/153 [==============================] - 1s 4ms/step - loss: 0.7140 - accuracy: 0.8760 - val_loss: 0.4000 - val_accuracy: 0.8790
Epoch 4/50
153/153 [==============================] - 1s 4ms/step - loss: 0.6344 - accuracy: 0.8680 - val_loss: 0.3843 - val_accuracy: 0.8790
Epoch 5/50
153/153 [==============================] - 1s 4ms/step - loss: 0.5077 - accuracy: 0.8767 - val_loss: 0.3830 - val_accuracy: 0.8790
Epoch 6/50
153/153 [==============================] - 1s 4ms/step - loss: 0.4755 - accuracy: 0.8768 - val_loss: 0.3858 - val_accuracy: 0.8790
Epoch 7/50
153/153 [==============================] - 1s 4ms/step - loss: 0.4585 - accuracy: 0.8768 - val_loss: 0.3742 - val_accuracy: 0.8790
E

In [13]:
pred=model.predict(test2[x.columns]).ravel()

In [17]:
pred=model.predict_proba(test2[x.columns])

Instructions for updating:
Please use `model.predict()` instead.


In [18]:
pred[:5]

array([[0.12317473],
       [0.12317473],
       [0.12317473],
       [0.12317473],
       [0.12317473]], dtype=float32)